In [84]:
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os

os.popen("adb shell dumpsys gfxinfo mobisocial.arcade reset")
os.system("adb shell input swipe 700 1800 700 200") 

command = "adb shell dumpsys gfxinfo mobisocial.arcade framestats | grep -A 120 'Flags'"
info_columns_name = ['Flags', 'IntendedVsync', 'Vsync', 'OldestInputEvent', 'NewestInputEvent', 
                             'HandleInputStart', 'AnimationStart', 'PerformTraversalsStart', 'DrawStart', 
                             'SyncQueued', 'SyncStart', 'IssueDrawCommandsStart', 'SwapBuffers', 'FrameCompleted', 
                             'DequeueBufferDuration', 'QueueBufferDuration', 'NA']

result_columns_name = [' UI 线程中发生的工作使其无法及时响应垂直同步信号','应用处理输入事件所花的时间（ >2 毫秒表示处理输入事件时间长）',
                '正在运行的所有动画（ObjectAnimator、ViewPropertyAnimator 和通用转换）所需的时间',
                '完成布局和测量阶段所需的时间','对树中的所有视图调用 View.draw() 所需的时间',
                '约 > 0.4 毫秒表示绘制了大量必须上传到 GPU 的新位图','GPU 工作量','处理此帧所花的总时间','达标线']

result = os.popen(command).readlines()
result.pop(0)

info_df = pd.DataFrame(columns = info_columns_name)

i = 0
for line in result:
        eachline = line.split(',')
        if (len(eachline) == len(info_columns_name)) :
            series = pd.Series(eachline, dtype = "str", index = info_columns_name)
            info_df.loc[i] = series 
        i += 1
    
# info_df.head()


result_df = pd.DataFrame(columns = result_columns_name)
result_df.



,Flags,IntendedVsync,Vsync,OldestInputEvent,NewestInputEvent,HandleInputStart,AnimationStart,PerformTraversalsStart,DrawStart,SyncQueued,SyncStart,IssueDrawCommandsStart,SwapBuffers,FrameCompleted,DequeueBufferDuration,QueueBufferDuration,NA
0,0,176016358084525,176016358084525,176016348000000,176016353084525,176016359689156,176016363381968,176016363743114,176016365866239,176016366190823,176016366657854,176016367401500,176016373476552,176016376151291,617000,1643000,\n
1,0,176016374766018,176016374766018,176016363000000,176016369766018,176016375249937,176016376268791,176016376489833,176016376643479,176016376790302,176016377011083,176016377303114,176016380185875,176016384621239,202000,3869000,\n
2,0,176016391447093,176016391447093,176016376000000,176016382500000,176016393079885,176016393815718,176016393954468,176016394056135,176016394174364,176016394261656,176016394523375,176016402325823,176016403374989,1884000,389000,\n
3,0,176016408124053,176016408124053,176016394000000,176016402000000,176016409835770,176016410534937,176016410676604,176016410782333,176016410891083,176016411145458,176016411422073,176016417490718,176016418891500,717000,660000,\n
4,0,176016424801941,176016424801941,176016411000000,176016419000000,176016426463948,176016427465041,176016427604364,176016427703687,176016427819573,176016427908375,176016428207489,176016431898739,176016432751343,233000,349000,\n


In [ ]:
import time
import sys
import os
import openpyxl
from openpyxl import Workbook
from openpyxl.styles.colors import Color
from openpyxl.chart import LineChart, Reference, Series

from copy import deepcopy

titlelist = [' UI 线程中发生的工作使其无法及时响应垂直同步信号','应用处理输入事件所花的时间（ >2 毫秒表示处理输入事件时间长）',
                '正在运行的所有动画（ObjectAnimator、ViewPropertyAnimator 和通用转换）所需的时间',
                '完成布局和测量阶段所需的时间','对树中的所有视图调用 View.draw() 所需的时间',
                '约 > 0.4 毫秒表示绘制了大量必须上传到 GPU 的新位图','GPU 工作量','处理此帧所花的总时间','达标线']
subtitlelist = ['IntendedVsync','HandleInputStart','AnimationStart','PerformTraversalsStart',
                'DrawStart','SyncStart','IssueDrawCommandsStart','FrameCompleted','Avg']

# 輸出資料表名稱
titlename = "Feed"
print ("Starting")

# 統計圖資料表名稱
statname = "Stats"
statwb = Workbook()
statws = statwb.active
statws.title = "statistic"
statws.column_dimensions["A"].width = 12
statws.column_dimensions["B"].width = 12
statws.column_dimensions["C"].width = 12
statws.column_dimensions["D"].width = 12
statws.column_dimensions["E"].width = 12
statws.column_dimensions["F"].width = 12
statws.column_dimensions["G"].width = 12
statws.column_dimensions["H"].width = 12
statws.column_dimensions["I"].width = 12


# 執行 3 次
for j in range(1,4):
    time.sleep(1)
    print ("開始執行第 " + str(j) + " 次")

    wb = Workbook()
    ws = wb.active
    ws.title = "data"
    valueofwidth = 16
    ws.column_dimensions["A"].width = valueofwidth
    ws.column_dimensions["B"].width = valueofwidth
    ws.column_dimensions["C"].width = valueofwidth
    ws.column_dimensions["D"].width = valueofwidth
    ws.column_dimensions["E"].width = valueofwidth
    ws.column_dimensions["F"].width = valueofwidth
    ws.column_dimensions["G"].width = valueofwidth
    ws.column_dimensions["H"].width = valueofwidth
    ws.column_dimensions["I"].width = valueofwidth
    ws.column_dimensions["J"].width = valueofwidth
    ws.column_dimensions["K"].width = valueofwidth
    ws.column_dimensions["L"].width = valueofwidth
    ws.column_dimensions["M"].width = valueofwidth
    ws.column_dimensions["N"].width = valueofwidth

    # 重置
    os.popen("adb shell dumpsys gfxinfo mobisocial.arcade reset")
    print ("清理幀資料")

    # 模擬滑動換頁操作
    for i in range (1,3):
        print ("執行滑動換頁第 " + str(i) + " 次")
        os.system("adb shell input swipe 700 1800 700 200") 
        time.sleep(1)

    # 过滤、筛选精确的帧时间信息
    command = "adb shell dumpsys gfxinfo mobisocial.arcade framestats | grep -A 120 'Flags'"
    r = os.popen(command)
    info = r.readlines()

    # 数据处理中
    print ("缓存数据中......")
    for line in info:  #按行遍历
        # line = line.strip('\r\n')
        eachline = line.split(',')
        # 将行写入Excel表格
        ws.append(eachline)
        # print line

    # 新建sheet用来统计数据
    resultsheet = wb.create_sheet("result",0)
    resultsheet.column_dimensions["A"].width = valueofwidth
    resultsheet.column_dimensions["B"].width = valueofwidth
    resultsheet.column_dimensions["C"].width = valueofwidth
    resultsheet.column_dimensions["D"].width = valueofwidth
    resultsheet.column_dimensions["E"].width = valueofwidth
    resultsheet.column_dimensions["F"].width = valueofwidth
    resultsheet.column_dimensions["G"].width = valueofwidth
    resultsheet.column_dimensions["H"].width = valueofwidth
    resultsheet.column_dimensions["I"].width = valueofwidth

    # 为结果页添加title说明
    resultsheet.append(titlelist)
    resultsheet.append(subtitlelist)
    # resultsheet.RowDimension(height = 5)

    # 填入公式，cell值由纳秒转换为毫秒
    for i in range(3,123):
        resultsheet.cell(row = i, column = 1, value = "=data!C" + str(i-1) + "-data!B"+ str(i-1))

    for i in range(3,123):
        value = "=(data!G" + str(i-1) + "-data!F"+ str(i-1)
        resultsheet.cell(row = i, column = 2, value = value + ")/1000000")

    for i in range(3,123):
        value = "=(data!H" + str(i-1) + "-data!G"+ str(i-1)
        resultsheet.cell(row = i, column = 3, value = value + ")/1000000")

    for i in range(3,123):
        value = "=(data!I" + str(i-1) + "-data!G"+ str(i-1)
        resultsheet.cell(row = i, column = 4, value = value + ")/1000000")

    for i in range(3,123):
        value = "=(data!K" + str(i-1) + "-data!I"+ str(i-1)
        resultsheet.cell(row = i, column = 5, value = value + ")/1000000")

    for i in range(3,123):
        value = "=(data!L" + str(i-1) + "-data!K"+ str(i-1)
        resultsheet.cell(row = i, column = 6, value = value + ")/1000000")

    for i in range(3,123):
        value = "=(data!L" + str(i-1) + "-data!K"+ str(i-1)
        resultsheet.cell(row = i, column = 7, value = value + ")/1000000")

    for i in range(3,123):
        value = "=(data!N" + str(i-1) + "-data!B"+ str(i-1)
        resultsheet.cell(row = i, column = 8, value = value + ")/1000000")

    # 插入平均值16ms的列
    for i in range(3,123):
        resultsheet.cell(row = i, column = 9, value = 16)

    # 插入平均Frame值
    resultsheet['J1'] = "平均值ms"
    resultsheet['J2'] = "=AVERAGEA(H3:H122)"

    # 画图准备
    chart = LineChart()
    chart.title = titlename + str(j)
    # chart.style = 5       #style都很丑，还不如默认的
    chart.y_axis.title = 'ms'
    chart.x_axis.title = 'Frame'
    chart.width = 30
    chart.height = 15

    # data选取范围
    data = Reference(resultsheet, min_col=8, min_row=2, max_col=9, max_row=122)
    chart.add_data(data, titles_from_data=True)
    
    copychart = deepcopy(chart)

    # 创建图表,在B3位置插入
    resultsheet.add_chart(chart,"B3")
    statws.add_chart(copychart,"B3")

    #记录时间戳作为文件名
    # filename = time.strftime('%Y%m%d_%H%M%S',time.localtime(time.time())) + ".xlsx"
    # wb.save(filename)

    #以执行名称 titlename作为文件名
    filename2 = titlename + str(j) + ".xlsx"
    wb.save(filename2)
    
    statwb.save(statname + ".xlsx")

    # 数据完毕
    print ("缓存处理完毕，保存数据到本地" + str(filename2))
    time.sleep(3)
    
print ("執行完畢")